## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn import metrics
import seaborn as sns
from scipy import stats, integrate
import matplotlib.pyplot as plt
import sys,  datetime, os

## Load Data

In [ ]:
X2 = pd.read_table('Output/Morg2f_BMatrix_map_2018_07.tsv')
X2[X2.columns[0]] = X2[X2.columns[0]].astype(str)
X2.set_index(X2.columns[0], inplace = True)
# X2= X2.T
X2.head(1)

In [ ]:
X1 = pd.read_table('All_bmat/DrugBank_Targets_2018_06.tsv')
X1.set_index(X1.columns[0], inplace = True)
X1 = X1.T
X1.head(1)

## Get only drugs shared by the two dfs

In [ ]:
shared_drugs = sorted(list(set(X1.index) & set(X2.index)))
X1 = X1.loc[shared_drugs]
X2 = X2.loc[shared_drugs]
print(X1.shape, X2.shape)

## Eliminate Sparse Rows and Columns  then create Similarity Matrix

In [ ]:
sns.distplot(list(X1.sum(axis=0)))

In [ ]:
x1_col_drop = []
x1_amt_nonzero = X1.sum(axis=0)
for amt in x1_amt_nonzero.iteritems():
    if abs(amt[1]) < 3:
        x1_col_drop.append(amt[0])
X1 = X1.drop(columns = x1_col_drop)
X1.head()

In [ ]:
sns.distplot(list(X2.sum(axis=0)))

In [ ]:
x2_col_drop = []
x2_amt_nonzero = X2.sum(axis=0)
for amt in x2_amt_nonzero.iteritems():
    if amt[1] <21:
        x2_col_drop.append(amt[0])
        
X2 = X2.drop(columns = x2_col_drop)
X2.shape

In [ ]:
shared_drugs = sorted(list(set(X1.index) & set(X2.index)))
X1 = X1.loc[shared_drugs]
X2 = X2.loc[shared_drugs]
print(X1.shape, X2.shape)

In [ ]:
sim_mat = 1- metrics.pairwise.pairwise_distances(X1.values.T, X2.values.T, metric='cosine')
sns.heatmap(pd.DataFrame(sim_mat, index=X1.columns, columns=X2.columns))

In [ ]:
clustermap = sns.clustermap(pd.DataFrame(sim_mat, index=X1.columns, columns=X2.columns), cmap="jet")

## Save the df and clustermap

In [ ]:
clustermap.savefig('Clustermaps/l1000_topt.png')
filename = 'Clustermaps/Morg2f_drugbank.tsv'### % str(datetime.date.today())[0:7].replace('-', '_')
sim_df = pd.DataFrame(sim_mat, index=X1.columns, columns=X2.columns)
sim_df.to_csv(filename, sep='\t')